<a href="https://colab.research.google.com/github/alirezash97/BraTS/blob/master/results_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !wget https://www.cbica.upenn.edu/MICCAI_BraTS2020_ValidationData

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# !unzip /content/MICCAI_BraTS2020_ValidationData -d '/content/drive/My Drive/BraTS2020 validation/'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:

import os
import numpy as np
from nibabel.testing import data_path
import nibabel as nib
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import cv2
import keras 
import random

In [6]:
from keras import backend as K

def soft_dice_loss(y_true, y_pred, axis=(1, 2, 3), 
                   epsilon=0.00001):
  
    dice_numerator = 2. * K.sum(y_true * y_pred, axis=axis) + epsilon
    dice_denominator = K.sum(y_true**2, axis=axis) + K.sum(y_pred**2, axis=axis) + epsilon
    dice_loss = 1 - K.mean((dice_numerator)/(dice_denominator))


    return dice_loss


def dice_coefficient(y_true, y_pred, axis=(1, 2, 3), 
                     epsilon=0.00001):

    dice_numerator = 2. * K.sum(y_true * y_pred, axis=axis) + epsilon
    dice_denominator = K.sum(y_true, axis=axis) + K.sum(y_pred, axis=axis) + epsilon
    dice_coefficient = K.mean((dice_numerator)/(dice_denominator))
    
 
    return dice_coefficient

In [7]:

# load and evaluate a saved model
from numpy import loadtxt
from keras.models import load_model
 
# load model
model = load_model('/content/drive/My Drive/BRATS2020/newerby_model.01-0.33.h5', custom_objects={'soft_dice_loss':soft_dice_loss, 'dice_coefficient':dice_coefficient})
# summarize model.
model.summary()
# load dataset

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 120, 120, 16 0                                            
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, 120, 120, 16, 3488        input_1[0][0]                    
__________________________________________________________________________________________________
conv3d_14 (Conv3D)              (None, 120, 120, 16, 3488        input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 120, 120, 16, 0           conv3d[0][0]                     
_______________________________________________________________________________________

In [8]:
import glob, os
images_path = glob.glob('/content/drive/My Drive/BRATS2020/**/*.nii.gz', recursive=True)

data_path = ""

X_trainset_filenames = []
y_trainset_filenames = []

for item in images_path:
  if 'seg' in item:
    y_trainset_filenames.append(os.path.join(data_path, item))
  else:
    X_trainset_filenames.append(os.path.join(data_path, item))

In [9]:
print(len(X_trainset_filenames))

1476


In [10]:
for i in range (300, 304):
  print(X_trainset_filenames[i])

/content/drive/My Drive/BRATS2020/MICCAI_BraTS2020_TrainingData/BraTS20_Training_085/BraTS20_Training_085_t1.nii.gz
/content/drive/My Drive/BRATS2020/MICCAI_BraTS2020_TrainingData/BraTS20_Training_085/BraTS20_Training_085_flair.nii.gz
/content/drive/My Drive/BRATS2020/MICCAI_BraTS2020_TrainingData/BraTS20_Training_085/BraTS20_Training_085_t1ce.nii.gz
/content/drive/My Drive/BRATS2020/MICCAI_BraTS2020_TrainingData/BraTS20_Training_085/BraTS20_Training_085_t2.nii.gz


In [11]:
def load_case(image_nifty_file, label_nifty_file):


    image_nifty_file 
    
    # load the image and label file, get the image content and return a numpy array for each
    image = np.zeros((240, 240, 155, 4))

    img0 = np.array(nib.load(image_nifty_file[0]).get_fdata())
    img1 = np.array(nib.load(image_nifty_file[1]).get_fdata())
    img2 = np.array(nib.load(image_nifty_file[2]).get_fdata())
    img3 = np.array(nib.load(image_nifty_file[3]).get_fdata())
    
    image[:, :, :, 0] = img0
    image[:, :, :, 1] = img1
    image[:, :, :, 2] = img2
    image[:, :, :, 3] = img3
      
    label = np.array(nib.load(label_nifty_file).get_fdata())
    
    
    return image, label

In [12]:
def load_case_validation(image_nifty_file):


    image_nifty_file 
    
    # load the image and label file, get the image content and return a numpy array for each
    image = np.zeros((240, 240, 155, 4))

    img0 = np.array(nib.load(image_nifty_file[0]).get_fdata())
    img1 = np.array(nib.load(image_nifty_file[1]).get_fdata())
    img2 = np.array(nib.load(image_nifty_file[2]).get_fdata())
    img3 = np.array(nib.load(image_nifty_file[3]).get_fdata())
    
    image[:, :, :, 0] = img0
    image[:, :, :, 1] = img1
    image[:, :, :, 2] = img2
    image[:, :, :, 3] = img3
      
    # label = np.array(nib.load(label_nifty_file).get_fdata())
    
    
    return image #, label

In [13]:
def sort_by_channel(sample_path):

  n = int(len(sample_path) / 4)
  new_path = []
  for i in range(n):
    temp = sample_path[(i*4): (i+1)*4]
    new_temp = []

    ###############
    for path in temp:
      if '_t1.' in path:
        new_temp.append(path)
      else:
        pass
    for path in temp:
      if '_t1ce.' in path:
        new_temp.append(path)
      else:
        pass
    for path in temp:
      if '_t2.' in path:
        new_temp.append(path)
      else:
        pass
    for path in temp:
      if '_flair.' in path:
        new_temp.append(path)
      else:
        pass

    ################    
    for path in new_temp:
      new_path.append(path)


  return new_path

In [14]:
X_trainset_filenames_by_channels = sort_by_channel(X_trainset_filenames[100:104])

In [15]:
image , label = load_case(X_trainset_filenames_by_channels, y_trainset_filenames[25])
# image = load_case_validation(X_trainset_filenames_by_channels)

In [16]:
print(image.shape)
# print(label.shape)

(240, 240, 155, 4)


In [17]:
def get_sub_volume(image, label):
  
  sub_volume_X = []
  sub_volume_y = []
  sub_volume_X_middle = []

  for z in range(8, 147, 16):
    
    sub_middle_0 = image[60:180, :120, z::z+16, :]
    sub_middle_1 = image[60:180, 120:, z::z+16, :] 
    for j in range (0, 240, 120):
      for k in range(0, 240, 120):
      

        sub = image[ j:j+120, k:k+120, z:z+16, :]
        sub_volume_X.append(sub)
        sub = label[ j:j+120, k:k+120, z:z+16]
        sub_volume_y.append(sub)
    
    sub_volume_X_middle.append(sub_middle_0)
    sub_volume_X_middle.append(sub_middle_1)




  return sub_volume_X, sub_volume_y, sub_volume_X_middle





In [18]:
def get_sub_volume_validation(image, label):
  
  sub_volume_X = []
  sub_volume_X_middle = []
  sub_volume_y = []

    
  for z in range(8, 147, 16):

    sub_middle_0 = image[:120, 60:180, z:z+16, :]
    sub_middle_1 = image[120:, 60:180, z:z+16, :]       
    for j in range(0, 240, 120):
      for k in range(0, 240, 120):
      
        sub = image[ j:j+120, k:k+120, z:z+16, :]
        sub_volume_X.append(sub)
        sub = label[ j:j+120, k:k+120, z:z+16]
        sub_volume_y.append(sub)
    sub_volume_X_middle.append(sub_middle_0)
    sub_volume_X_middle.append(sub_middle_1)



  return sub_volume_X, sub_volume_X_middle , sub_volume_y





In [19]:
# X, y = get_sub_volume(image, label)
X, X_middle, y = get_sub_volume_validation(image, label)

In [20]:
print(len(X))
print(len(X_middle))
# print(len(y))

36
18


In [21]:
print(X[1].shape)
print(X_middle[10].shape)
# print(y[1].shape)

(120, 120, 16, 4)
(120, 120, 16, 4)


In [22]:
from tqdm import tqdm

y_pred = []

for item in tqdm(X):
  temp_0 = item.reshape((1, 120, 120, 16, 4))
  temp = model.predict(temp_0)
  y_pred.append(temp)

y_pred_middle = []
for item in tqdm(X_middle):
  temp_0 = item.reshape((1, 120, 120, 16, 4))
  temp = model.predict(temp_0)
  y_pred_middle.append(temp)


100%|██████████| 18/18 [00:10<00:00,  1.66it/s]


In [23]:
y_pred_middle[0].shape

(1, 120, 120, 16, 4)

In [24]:
def get_whole_image(predict, predict_middle):
  
  prediction = np.zeros((240, 240, 155, 4))

  counter = 0
  counter_middle = 0
  for z in range(8, 147, 16):  
    for j in range (0, 240, 120):
      for k in range(0, 240, 120):
      
        prediction[j:j+120, k:k+120, z:z+16, :] = predict[counter]
        counter +=1

    middle_temp_0 = np.reshape(predict_middle[counter_middle], (120, 120, 16, 4))
    prediction[:120, 60:120, z:z+16, :] += middle_temp_0[:, :60, :, :]
    prediction[:120, 120:180, z:z+16, :] += middle_temp_0[:, 60:, :, :]

    middle_temp_1 = np.reshape(predict_middle[counter_middle+1], (120, 120, 16, 4))
    prediction[120:, 60:120, z:z+16, :] += middle_temp_1[:, :60, :, :]
    prediction[120:, 120:180, z:z+16, :] += middle_temp_1[:, 60:, :, :]
    counter_middle += 2

        



  return prediction

In [25]:
y = get_whole_image(y_pred, y_pred_middle)
print(y.shape)

(240, 240, 155, 4)


In [26]:
# from ipywidgets import interact

# def explore_3dimage(layer):
#     plt.figure(figsize=(10, 5))
#     channel = 3
#     tt = X_middle[12]
#     plt.imshow(tt[:, :, layer, 3]);
#     plt.title('Explore Layers of MR image', fontsize=20)
#     plt.axis('off')
#     return layer

# # Run the ipywidgets interact() function to explore the data
# interact(explore_3dimage, layer=(0, 16 - 1));

In [27]:
from ipywidgets import interact

def explore_3dimage(layer):
    plt.figure(figsize=(10, 5))
    channel = 3
    plt.imshow(image[:, :, layer, 1]);
    plt.title('Explore Layers of MR image', fontsize=20)
    plt.axis('off')
    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_3dimage, layer=(0, 155 - 1));

interactive(children=(IntSlider(value=77, description='layer', max=154), Output()), _dom_classes=('widget-inte…

In [28]:
print(y.shape)

(240, 240, 155, 4)


In [29]:
from ipywidgets import interact

def explore_3dimage(layer):
    plt.figure(figsize=(10, 5))
    channel = 3
    # plt.imshow(y[:, :, layer, :]);
    plt.imshow((y[:, :, layer, 1:4] * 255).astype(np.uint8));
    plt.title('Explore Layers of Tumor prediction', fontsize=20)
    plt.axis('off')
    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_3dimage, layer=(0, 155 - 1));

interactive(children=(IntSlider(value=77, description='layer', max=154), Output()), _dom_classes=('widget-inte…

In [30]:
# y[140:145, 140:145, 90, 0]

In [31]:
# valid_y[140:145, 140:145, 90]

In [ ]:
valid_y = np.zeros((240, 240, 155))

for i in range(240):
  for j in range(240):
    for k in range(155):

      if y[i, j, k, 0] != 0.0:
        valid_y[i, j, k] = y[i, j, k, 0]
      
      elif y[i, j, k, 1] != 0.0:
        valid_y[i, j, k] = y[i, j, k, 1]

      elif y[i, j, k, 2] != 0.0:
        valid_y[i, j, k] = y[i, j, k, 2]

      elif y[i, j, k, 3] != 0.0:
        valid_y[i, j, k] = y[i, j, k, 2]

      else:
        valid_y[i, j, k] = 0. 


# valid_y = np.argmax(y, axis=-1)


from ipywidgets import interact

def explore_3dimage(layer):
    plt.figure(figsize=(10, 5))
    channel = 3
    # plt.imshow(y[:, :, layer, :]);
    plt.imshow((valid_y[:, :, layer] * 255).astype(np.uint8));
    plt.title('Explore Layers of Tumor prediction', fontsize=20)
    plt.axis('off')
    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_3dimage, layer=(0, 155 - 1));

In [ ]:
from ipywidgets import interact

def explore_3dimage(layer):
    plt.figure(figsize=(10, 5))
    channel = 3
    plt.imshow(label[:, :, layer]);
    plt.title('Explore Layers of Tumor ground truth', fontsize=20)
    plt.axis('off')
    return layer

# Run the ipywidgets interact() function to explore the data
interact(explore_3dimage, layer=(0, 155 - 1));